## Connect to ICW:

In [1]:
import cadspy

In [2]:
username = str(input('User number'))
icw = cadspy.DatabaseConnection(system='ICW', user=username)

<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:salmon;font-family:Verdana,sans-serif;font-size:16px;">

<font size="4">Can't connect to ICW? Instructions on how to get access to it in the links below:

</font>
</div>

- Step 1: [Requesting Access to GitHub](https://baplc.sharepoint.com/sites/ask/SitePages/Requesting-Access-to-GitHub.aspx)

Once your access to GitHub has been aproved, you need to:

- Step 2: [Request access to British-Ent GitHub organisation](https://github.com/BritishAirways-Ent/insight-processes/blob/main/onboarding/Corporate_Directory_git.md)

And

- Step 3: [Setup Sagemaker Studio to access GitHub](https://github.com/BritishAirways-Ent/insight-processes/blob/main/onboarding/sagemaker_to_github_setup.md)

<br>

Below are some packages to get you started. You don't have to use them but you may find them useful!

In [3]:
import numpy as np
import pandas as pd
import datetime as dt

In [4]:
# diplay all rows and cols when using 'dataframe'.head() or 'dataframe'.tail()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

<br>

### Data

#### S19 Lounge Eligibility data

In [5]:
query = """

sel * from LDB_SBOX_OR.HACKATHON_OPS_LOUNGE_ELIGIBILITY

"""

df_lounge_eligibility = icw.queryToDataframe(query)

In [6]:
df_lounge_eligibility.head(2)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13


In [7]:
df_lounge_eligibility.shape

(1534750, 12)

In [8]:
# a look to a particular flight number and date

#mask = (df_lounge_eligibility['DISCHARGE_STN_CD'] == 'GCM   ') & (df_lounge_eligibility['GMT_UPLIFT_DT'] == dt.date(2019,9,12) )

#df_lounge_eligibility[mask]

#### S19 Flight info

In [9]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_FLIGHT_INFO

"""

df_flight_info = icw.queryToDataframe(query)

In [10]:
df_flight_info.head(2)
df_flight_info.dtypes

GMT_PLND_DEP_TS    datetime64[ns]
GMT_ACT_DEP_TS     datetime64[ns]
OPG_ALN_CD                 object
OPG_FLT_NO                  int64
ACT_DEP_STN_CD             object
ACT_DEP_TML_CD             object
PLND_ARR_STN_CD            object
ACT_ARR_STN_CD             object
IATA_AC_TYP_CD             object
ACT_AC_TYP_CD              object
ROUTE                      object
dtype: object

#### Station Code Decode

In [11]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_COUNTRY_DECODE

"""

df_country = icw.queryToDataframe(query)

In [12]:
df_country.head(2)

,ROUTE,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM
0,LHRINV,GB,United Kingdom and Northern Ireland,UK,UK
1,LHRSVO,RU,Russia in Europe,EASTERN EUROPE,EUROPE EXC UK


#### Aircraft Type

In [13]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_AC_TYPE

"""

df_acft_typ = icw.queryToDataframe(query)

In [14]:
df_acft_typ.head(2)

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,320,A3,NB,0,24,0,132
1,777,K7,WB,0,48,24,203


In [15]:
df_acft_typ.shape

(20, 7)

In [16]:
df_acft_typ

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,320,A3,NB,0,24,0,132
1,777,K7,WB,0,48,24,203
2,32N,N3,NB,0,20,0,150
3,321,M6,NB,0,23,0,131
4,32Q,N6,NB,0,32,0,172
5,789,L8,WB,8,42,39,127
6,32A,H3,NB,0,20,0,150
7,319,A4,NB,0,16,0,119
8,788,B8,WB,0,35,25,154
9,77W,G7,WB,14,56,44,183


Merging df_country with route haul and business/leisure data from reference tables in ICW.

They will be merged into df_lounge_eligibility for further analysis.

In [17]:
# A corporate_reference table just for flight numbers
# and their descriptions.
# To be matched on flight number and date (inner join probably)
# Checked with TLV and AMM, both correct (LH, SH)
query = """

select FM_FLT_NO, TO_FLT_NO, EFF_DT, EXP_DT, LONGHAUL_IND, SHORTHAUL_IND,
DOMESTIC_IND, RANGE_DESC_TXT

from CRT_FLIGHT_NO_RANGES

WHERE CODESHARE_IND = 'N' AND FRANCHISE_IND = 'N' 

"""

df_flt_no_ranges = icw.queryToDataframe(query)

In [18]:
df_flt_no_ranges.head(10)

,FM_FLT_NO,TO_FLT_NO,EFF_DT,EXP_DT,LONGHAUL_IND,SHORTHAUL_IND,DOMESTIC_IND,RANGE_DESC_TXT
0,8000,8019,2019-03-31,2021-08-12,Y,N,N,Openskies for LEVEL France
1,2530,2899,2022-10-30,2022-10-31,N,Y,N,LGW Euroflyer Prime Shorthaul
2,2530,2825,2023-10-18,2023-10-18,N,Y,N,LGW Euroflyer Prime LGW Shorthaul
3,2360,2369,2022-10-30,2022-10-31,N,Y,N,BA Cityflyer Prime
4,3000,3199,2005-01-01,2021-10-30,N,N,N,BA Cargo Interline Services (Robbie Newport x8...
5,2826,2827,2023-10-18,2023-10-18,N,Y,N,LGW Euroflyer Prime LGW Shorthaul
6,2210,2217,2019-03-31,2021-10-30,N,Y,N,BA Cityflyer (CJ) (London City services Shorth...
7,3300,3319,2005-01-01,2021-10-30,Y,N,N,BA wet lease and shared longhaul freighter ser...
8,9278,9279,2022-03-27,2022-10-29,N,N,N,Euroflyer Engineering
9,2218,2229,2019-03-31,2021-10-30,N,Y,N,BA Cityflyer (CJ) (London City Services)


In [19]:
# For more granular details of route type, we can also use RevMan definitions.

query = """

SELECT DEP_STN_CD, ARR_STN_CD, GCD_KM, REVMAN_HUB_TXT, 
REVMAN_AREA_TXT, ROUTE_CAT_TXT
FROM REF_ROUTE

WHERE DEP_STN_CD = 'LHR' AND REVMAN_HUB_TXT <> 'Unknown'

"""

df_route_categories = icw.queryToDataframe(query)

<br>

### Pre-processing

*Hint:* It is always worth checking the format of each of the columns in your dataframes before trying to do any work with them. To do so, you can make use of the `headers_and_first_row` function below.


In [20]:
def headers_and_first_row(df):
    '''
    print headers and first row of a df to deal with data types
    '''
    
    headers = df.columns
    first_row = []

    for col in headers:
        first_row.append(df[col][0])
    
    dictionary = dict( zip( headers, first_row) )

    return dictionary

In [21]:
# applying headers_and_first_row to df_lounge_eligibility
format_df = headers_and_first_row(df_lounge_eligibility)
format_2 = headers_and_first_row(df_flight_info)

In [22]:
# Note that some columns have blank spaces!
format_df
format_2

{'GMT_PLND_DEP_TS': Timestamp('2023-04-30 20:25:00'),
 'GMT_ACT_DEP_TS': Timestamp('2023-04-30 20:34:00'),
 'OPG_ALN_CD': 'BA    ',
 'OPG_FLT_NO': 57,
 'ACT_DEP_STN_CD': 'LHR   ',
 'ACT_DEP_TML_CD': '5 ',
 'PLND_ARR_STN_CD': 'JNB   ',
 'ACT_ARR_STN_CD': 'JNB   ',
 'IATA_AC_TYP_CD': '388   ',
 'ACT_AC_TYP_CD': 'A8  ',
 'ROUTE': 'LHRJNB      '}

<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">**Exercise 1:** Pre-process **all** the tables above (df_lounge_eligibility, df_flight_info, df_country, df_acft_typ).

</font>

</ol>
</div>

We will first strip all frames of spaces. We will then go through each frame to properly prepare it.

We will then load some additional datasets that will be useful for analysis.

In [23]:
# Stripping strings

tables = [df_lounge_eligibility,df_flight_info,df_country,df_acft_typ,df_flt_no_ranges,df_route_categories]

# Define a function that fine all string fields and remove all blak spaces
def data_cleaning_string(df):
    # Get names of all fields in a dataframe
    fields = df.columns
    # loop for all fields, if data type is string then remove blank spaces
    for f in fields:
        if type(df[f][0]) == str:
            df[f] = df[f].str.strip()
    return df

for i,t in enumerate(tables):
    tables[i] = data_cleaning_string(t)

In [24]:
# Removing duplucates of flight_info

# Sort the table
df_flight_info = df_flight_info.sort_values(['GMT_PLND_DEP_TS','OPG_FLT_NO','GMT_ACT_DEP_TS'])
# Remove all duplicates and keep the first rows of all duplicated
df_flight_info = df_flight_info.drop_duplicates(subset = ['OPG_FLT_NO','GMT_PLND_DEP_TS'],keep = "first")


In [25]:
# 2. Flight info
# Ian's code to clean dataset - identify latest departure date for
# duplicated flights and eliminate it.
df_flight_info['GMT_PLND_DEP_DT'] = df_flight_info['GMT_PLND_DEP_TS'].dt.date
df_flight_info['GMT_PLND_DEP_TIME'] = df_flight_info['GMT_PLND_DEP_TS'].dt.time

In [26]:
# 3. Country
df_country['DEP_STN_CD'] = df_country['ROUTE'].str.slice(0,3)
df_country['ARR_STN_CD'] = df_country['ROUTE'].str.slice(3,6)


In [27]:
# 4. Aircraft type
df_acft_typ.sort_values('FIRST_SEATS_QTY').tail(10)

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
11,343,W9,WB,0,45,0,212
13,777,L7,WB,0,48,24,203
5,789,L8,WB,8,42,39,127
19,777,W7,WB,12,48,32,127
10,744,S4,WB,14,86,30,145
15,744,V4,WB,14,52,36,235
18,777,V7,WB,14,48,40,124
12,388,A8,WB,14,97,55,303
9,77W,G7,WB,14,56,44,183
16,777,P7,WB,17,48,24,127


In [28]:
df_lounge_eligibility.shape

(1534750, 12)

In [29]:
# Joining flight_no_ranges with lounge eligibility
# df_merged_a = pd.merge(df_lounge_eligibility,df_flt_no_ranges,how='left',
#                       left_on=['OPERATING_FLT_NO'],right_on=['FM_FLT_NO'])
# print(df_merged_a.shape)
# df_merged_a = df_merged_a[(df_merged_a['GMT_UPLIFT_DT'] >= df_merged_a['EFF_DT'])&
#                     (df_merged_a['GMT_UPLIFT_DT']<=df_merged_a['EXP_DT'])]
# df_merged_a.drop(['FM_FLT_NO','TO_FLT_NO','EFF_DT','EXP_DT'],axis=1,inplace=True)
# print(df_merged_a.shape)

In [30]:
# df_merged_a.sort_values(['Skew_Id']).head(2)
# # success

# df_merged_a.shape

In [31]:
# df_merged_b = pd.merge(df_merged_a,df_route_categories,
#                       left_on=['UPLIFT_STN_CD','DISCHARGE_STN_CD'],
#                       right_on=['DEP_STN_CD','ARR_STN_CD'],
#                       how='left')

In [32]:
# # check for success
# df_merged_b.head(2)

# df_merged_b.shape

In [33]:
# df_lounge_eligibility = df_merged_b.copy()

# df_lounge_eligibility.head(5)

# df_lounge_eligibility.shape

<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">**Exercise 2:** Join the tables below
   
    - df_flight_info
    - df_country
    - df_acft_typ
    
to the table df_lounge_eligibility to generate a final dataset.

</font>

</ol>
</div>

PLUS for us, join df_flight_no_ranges and df_route_categories.

In [79]:
# df_lounge_eligibility.head(5)

In [80]:
df_lounge_elig_flight_info = pd.merge(df_lounge_eligibility,# left table
                                     df_flight_info, # right table
                                     left_on = ['OPERATING_AIRLINE_CD','OPERATING_FLT_NO','GMT_UPLIFT_DT'], # left on? e.g. which columns from the left table are you joining on to?
                                     right_on = ['OPG_ALN_CD','OPG_FLT_NO','GMT_PLND_DEP_DT'] , # right on? # left on? e.g. which columns from the right table are you joining on to?
                                     how = "inner" # how? e.g. left, right, inner,etc
                                     )

df_lounge_elig_flight_info.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00
1,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold Guest List,,Tier 1,1,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00
2,2023-08-05618,BA,618,2023-08-05,LHR,OLB,M,M,,,Not eligible,68,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00
3,2023-08-05618,BA,618,2023-08-05,LHR,OLB,M,M,Gold Guest List,EMER,Tier 1,4,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00
4,2023-08-05618,BA,618,2023-08-05,LHR,OLB,M,M,Silver,SAPP,Tier 3,6,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00


In [81]:
# your code here!
df_lounge_elig_flight_info.shape

(1533567, 25)

Get rows with any null values

In [82]:
df_lounge_elig_flight_info[df_lounge_elig_flight_info.isna().any(axis=1)].head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME


In [83]:
df_flight_info[df_flight_info['OPG_FLT_NO'] == 8642], headers_and_first_row(df_flight_info)

(Empty DataFrame
 Columns: [GMT_PLND_DEP_TS, GMT_ACT_DEP_TS, OPG_ALN_CD, OPG_FLT_NO, ACT_DEP_STN_CD, ACT_DEP_TML_CD, PLND_ARR_STN_CD, ACT_ARR_STN_CD, IATA_AC_TYP_CD, ACT_AC_TYP_CD, ROUTE, GMT_PLND_DEP_DT, GMT_PLND_DEP_TIME]
 Index: [],
 {'GMT_PLND_DEP_TS': Timestamp('2023-04-30 20:25:00'),
  'GMT_ACT_DEP_TS': Timestamp('2023-04-30 20:34:00'),
  'OPG_ALN_CD': 'BA',
  'OPG_FLT_NO': 57,
  'ACT_DEP_STN_CD': 'LHR',
  'ACT_DEP_TML_CD': '5',
  'PLND_ARR_STN_CD': 'JNB',
  'ACT_ARR_STN_CD': 'JNB',
  'IATA_AC_TYP_CD': '388',
  'ACT_AC_TYP_CD': 'A8',
  'ROUTE': 'LHRJNB',
  'GMT_PLND_DEP_DT': datetime.date(2023, 4, 30),
  'GMT_PLND_DEP_TIME': datetime.time(20, 25)})

### Merge `df_country` to table

In [84]:
[headers_and_first_row(d) for d in [df_lounge_elig_flight_info,df_country]]

[{'Skew_Id': '2023-08-05618',
  'OPERATING_AIRLINE_CD': 'BA',
  'OPERATING_FLT_NO': 618,
  'GMT_UPLIFT_DT': datetime.date(2023, 8, 5),
  'UPLIFT_STN_CD': 'LHR',
  'DISCHARGE_STN_CD': 'OLB',
  'BOOKED_CABIN_CD': 'C',
  'TRAVEL_CABIN_CD': 'C',
  'BA_PAX_TIER': 'Gold For Life',
  'ONEWORLD_TIER': 'EMER',
  'Lounge_eligibility_tier': 'Tier 2',
  'pax': 2,
  'GMT_PLND_DEP_TS': Timestamp('2023-08-05 14:25:00'),
  'GMT_ACT_DEP_TS': Timestamp('2023-08-05 14:57:00'),
  'OPG_ALN_CD': 'BA',
  'OPG_FLT_NO': 618,
  'ACT_DEP_STN_CD': 'LHR',
  'ACT_DEP_TML_CD': '3',
  'PLND_ARR_STN_CD': 'OLB',
  'ACT_ARR_STN_CD': 'OLB',
  'IATA_AC_TYP_CD': '319',
  'ACT_AC_TYP_CD': 'M4',
  'ROUTE': 'LHROLB',
  'GMT_PLND_DEP_DT': datetime.date(2023, 8, 5),
  'GMT_PLND_DEP_TIME': datetime.time(14, 25)},
 {'ROUTE': 'LHRINV',
  'COUNTRY_CD': 'GB',
  'COUNTRY_NM': 'United Kingdom and Northern Ireland',
  'CORP_GEOG_CTRY_GRP_NM': 'UK',
  'CORP_GEOG_CONTINENT_NM': 'UK',
  'DEP_STN_CD': 'LHR',
  'ARR_STN_CD': 'INV'}]

In [85]:
df_lounge_elig_flight_info_country = pd.merge(df_lounge_elig_flight_info,# left table
                                     df_country, # right table
                                     left_on = ['PLND_ARR_STN_CD'], # left on? e.g. which columns from the left table are you joining on to?
                                     right_on = ['ARR_STN_CD'] , # right on? # left on? e.g. which columns from the right table are you joining on to?
                                     how = "inner" # how? e.g. left, right, inner,etc
                                     )

# df_lounge_elig_flight_info = df_lounge_elig_flight_info.drop_duplicates()

df_lounge_elig_flight_info_country.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB
1,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold Guest List,,Tier 1,1,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB
2,2023-08-05618,BA,618,2023-08-05,LHR,OLB,M,M,,,Not eligible,68,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB
3,2023-08-05618,BA,618,2023-08-05,LHR,OLB,M,M,Gold Guest List,EMER,Tier 1,4,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB
4,2023-08-05618,BA,618,2023-08-05,LHR,OLB,M,M,Silver,SAPP,Tier 3,6,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB


In [86]:
df_lounge_elig_flight_info_country.shape

(1486074, 32)

Check for null values

In [87]:
df_lounge_elig_flight_info_country[df_lounge_elig_flight_info_country.isna().any(axis=1)].head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD


In [88]:
df_country[df_country['ARR_STN_CD'] == 'OPO']

,ROUTE,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD


### `df_acft_typ`

In [89]:
df_lounge_elig_flight_info_country_acft_typ = pd.merge(
    df_lounge_elig_flight_info_country,
    df_acft_typ,
    left_on = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD'],
    right_on = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD'],
    how = 'left'
)

df_lounge_elig_flight_info_country_acft_typ.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN
1,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold Guest List,,Tier 1,1,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN
2,2023-08-05618,BA,618,2023-08-05,LHR,OLB,M,M,,,Not eligible,68,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN
3,2023-08-05618,BA,618,2023-08-05,LHR,OLB,M,M,Gold Guest List,EMER,Tier 1,4,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN
4,2023-08-05618,BA,618,2023-08-05,LHR,OLB,M,M,Silver,SAPP,Tier 3,6,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN


In [90]:
df_lounge_elig_flight_info_country_acft_typ.shape

(1486074, 37)

Check for null

In [91]:
df_lounge_elig_flight_info_country_acft_typ[df_lounge_elig_flight_info_country_acft_typ.isna().any(axis=1)].head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN
1,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold Guest List,,Tier 1,1,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN
2,2023-08-05618,BA,618,2023-08-05,LHR,OLB,M,M,,,Not eligible,68,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN
3,2023-08-05618,BA,618,2023-08-05,LHR,OLB,M,M,Gold Guest List,EMER,Tier 1,4,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN
4,2023-08-05618,BA,618,2023-08-05,LHR,OLB,M,M,Silver,SAPP,Tier 3,6,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN


In [92]:
df_acft_typ[df_acft_typ['ACT_AC_TYP_CD'] == 'M4']

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY


### More cleaning for columns redundancy

In [93]:
headers_and_first_row(df_lounge_elig_flight_info_country_acft_typ)

{'Skew_Id': '2023-08-05618',
 'OPERATING_AIRLINE_CD': 'BA',
 'OPERATING_FLT_NO': 618,
 'GMT_UPLIFT_DT': datetime.date(2023, 8, 5),
 'UPLIFT_STN_CD': 'LHR',
 'DISCHARGE_STN_CD': 'OLB',
 'BOOKED_CABIN_CD': 'C',
 'TRAVEL_CABIN_CD': 'C',
 'BA_PAX_TIER': 'Gold For Life',
 'ONEWORLD_TIER': 'EMER',
 'Lounge_eligibility_tier': 'Tier 2',
 'pax': 2,
 'GMT_PLND_DEP_TS': Timestamp('2023-08-05 14:25:00'),
 'GMT_ACT_DEP_TS': Timestamp('2023-08-05 14:57:00'),
 'OPG_ALN_CD': 'BA',
 'OPG_FLT_NO': 618,
 'ACT_DEP_STN_CD': 'LHR',
 'ACT_DEP_TML_CD': '3',
 'PLND_ARR_STN_CD': 'OLB',
 'ACT_ARR_STN_CD': 'OLB',
 'IATA_AC_TYP_CD': '319',
 'ACT_AC_TYP_CD': 'M4',
 'ROUTE_x': 'LHROLB',
 'GMT_PLND_DEP_DT': datetime.date(2023, 8, 5),
 'GMT_PLND_DEP_TIME': datetime.time(14, 25),
 'ROUTE_y': 'LHROLB',
 'COUNTRY_CD': 'IT',
 'COUNTRY_NM': 'Italy',
 'CORP_GEOG_CTRY_GRP_NM': 'MEDITERRANEAN',
 'CORP_GEOG_CONTINENT_NM': 'EUROPE EXC UK',
 'DEP_STN_CD': 'LHR',
 'ARR_STN_CD': 'OLB',
 'WB_NB_CAT': nan,
 'FIRST_SEATS_QTY': nan,
 

In [94]:
to_drop = ['GMT_PLND_DEP_TS','GMT_ACT_DEP_TS','OPG_ALN_CD','OPG_FLT_NO','ACT_DEP_STN_CD','PLND_ARR_STN_CD','ACT_ARR_STN_CD','GMT_UPLIFT_DT']

df_lounge_elig_flight_info_country_acft_typ = df_lounge_elig_flight_info_country_acft_typ.drop(columns = to_drop,)

df_lounge_elig_flight_info_country_acft_typ.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,ACT_DEP_TML_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,2023-08-05618,BA,618,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,3,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN
1,2023-08-05618,BA,618,LHR,OLB,C,C,Gold Guest List,,Tier 1,1,3,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN
2,2023-08-05618,BA,618,LHR,OLB,M,M,,,Not eligible,68,3,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN
3,2023-08-05618,BA,618,LHR,OLB,M,M,Gold Guest List,EMER,Tier 1,4,3,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN
4,2023-08-05618,BA,618,LHR,OLB,M,M,Silver,SAPP,Tier 3,6,3,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN


In [95]:
df_lounge_eligibility.isnull().values.any()

False

<br><br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">Exercise 3: Based on S2019/S2023 data, provide a lookup table of Lounge eligibility assumptions that can be applied to a future schedule. To do so, answer each of the following questions in the Markdown cell provided below. 

- What level of granularity do you use?
- What metric do you use to come up with Lounge eligibility profiles?

    
Note 1: **Provide evidence for your assumptions.** This can be in the form of tables, graphs, correlation matrix, etc.
    
Note 2: Make use of the examples below to give structure to your answer. Feel free to attend the Hackathon Clinics if you have any questions. 
</font>


    
</ol>
</div>

**Reasoning (Example 1)**:


Assume:
- Data has been preprocessed.
- Data has been joined, and a final dataset has been created. This dataset is the result of joining the 4 tables.

The final dataset has been called `df_lounge_elig_flight_info_country_acft_typ`.

<u>What level of granularity do you use?</u>
- I have decided to split all flights in the network based on their aircraft type. I will therefore have a lookup table with two categories: Narrowbody and Widebody. 

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- I have sumed up all the passengers by Aircraft Type, by Tier. Then I have divided them by the total number of passengers by Aircraft Type. For example:
    - For NB aircrafts, and for Tier 1 passengers: In S19 we had 41,728 pax eligible for Tier 1 out of 7,222,830 pax flying on Narrowbody aircraft. This represents 0.6% of the costumers and I assume that this will be the number of costumers elegible for this specific Lounge in a future schedule.   

In [96]:
# your code here!

In [97]:
# columns that you want to group by
list_groupby = ['WB_NB_CAT','Lounge_eligibility_tier']

# grouping by WB_NB_CAT and Lounge_eligibility_tier
df_groupby_wb_nb = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

In [98]:
# a look at the data
df_groupby_wb_nb

,WB_NB_CAT,Lounge_eligibility_tier,pax_count
0,NB,Not eligible,7052766
1,NB,Tier 1,40997
2,NB,Tier 2,556223
3,NB,Tier 3,1619091
4,WB,Not eligible,4132210
5,WB,Tier 1,197306
6,WB,Tier 2,283558
7,WB,Tier 3,1113149


In [99]:
# getting the number of pax by aircraft type

# columns that you want to group by
list_groupby = ['WB_NB_CAT']

# grouping by WB_NB_CAT
df_groupby_wb_nb_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

In [100]:
# a look at the data
df_groupby_wb_nb_ttl

,WB_NB_CAT,pax_count_ttl
0,NB,9269077
1,WB,5726223


In [101]:
# now, let's left join df_groupby_wb_nb_ttl onto df_groupby_wb_nb
# this adds a new column to the df_groupby_wb_nb table (pax_count_ttl) that will be used to get the percentage of passenger eligible by Tier

df_groupby_wb_nb = pd.merge(df_groupby_wb_nb,
                            df_groupby_wb_nb_ttl,
                            on = ['WB_NB_CAT'],
                            how = 'left'
                           )

In [102]:
# a look at the data
df_groupby_wb_nb

,WB_NB_CAT,Lounge_eligibility_tier,pax_count,pax_count_ttl
0,NB,Not eligible,7052766,9269077
1,NB,Tier 1,40997,9269077
2,NB,Tier 2,556223,9269077
3,NB,Tier 3,1619091,9269077
4,WB,Not eligible,4132210,5726223
5,WB,Tier 1,197306,5726223
6,WB,Tier 2,283558,5726223
7,WB,Tier 3,1113149,5726223


In [103]:
# getting the percentage of pax elegible for each of the Tiers

df_groupby_wb_nb['pax_eligible%'] = (df_groupby_wb_nb['pax_count'] / df_groupby_wb_nb['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_wb_nb['pax_eligible%'] = df_groupby_wb_nb['pax_eligible%'].map('{:,.1f}%'.format)

In [104]:
# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_wb_nb.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

In [105]:
# a look at the data
df_groupby_wb_nb

,WB_NB_CAT,Lounge_eligibility_tier,pax_eligible%
0,NB,Not eligible,76.1%
1,NB,Tier 1,0.4%
2,NB,Tier 2,6.0%
3,NB,Tier 3,17.5%
4,WB,Not eligible,72.2%
5,WB,Tier 1,3.4%
6,WB,Tier 2,5.0%
7,WB,Tier 3,19.4%


In [106]:
# dropping 'not eligible' rows - not needed anymore
mask = df_groupby_wb_nb['Lounge_eligibility_tier'] == 'Not eligible'

df_groupby_wb_nb = df_groupby_wb_nb[~mask].copy()

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.

In [107]:
# using set_index to come up with the final lookup table
df_groupby_wb_nb = df_groupby_wb_nb.set_index(['WB_NB_CAT','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

In [108]:
# final table
df_groupby_wb_nb

pax_eligible%              
Lounge_eligibility_tier        Tier 1 Tier 2 Tier 3
WB_NB_CAT                                          
NB                               0.4%   6.0%  17.5%
WB                               3.4%   5.0%  19.4%

<br>

**Feedback:** This analysis provides a lookup table in the format needed to be input into a future schedule. Nevertheless, this analysis is too high level and you haven't provided any evidence for your assumptions. To further enhance your answer use insights from the data and provide evidence for your assumptions. Please find some ideas below: 

- Using the same categories (WB,NB), plot data overtime to better understand the peaks for the different lounges. 
- Is there a way to split Widebody into more categories? Do the Haul, Region, Time of Day, or Country play a role in the number of passengers that are eligible in Tier 1, Tier 2 and Tier 3? Etc...

<br><br>

**Reasoning (Example 2)**:



<u>What level of granularity do you use?</u>
- I have decided to split all flights in the network based on their flight number. I will therefore have a lookup table with a lot of categories as each flight number is a category. 

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- I have sumed up all the passengers by flight number, by Tier. Then I have divided them by the total number of passengers flight number.  

In [109]:
# your code here!

In [110]:
# columns that you want to group by
list_groupby = ['OPERATING_FLT_NO','DISCHARGE_STN_CD','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_flt_no = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

# a look at the data
df_groupby_flt_no.head(4)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,Lounge_eligibility_tier,pax_count
0,5,HND,Not eligible,15825
1,5,HND,Tier 1,220
2,5,HND,Tier 2,888
3,5,HND,Tier 3,4469


In [111]:
# getting the number of pax by OPERATING_FLT_NO and DISCHARGE_STN_CD

# columns that you want to group by
list_groupby = ['OPERATING_FLT_NO','DISCHARGE_STN_CD']

# grouping by WB_NB_CAT
df_groupby_flt_no_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

In [112]:
# a look at the data
df_groupby_flt_no_ttl.head(2)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,pax_count_ttl
0,5,HND,21402
1,5,NRT,38378


In [113]:
# now, let's left join df_groupby_flt_no_ttl onto df_groupby_flt_no
# this adds a new column to the df_groupby_flt_no table (pax_count_ttl) that will be used to get the percentage of passenger eligible by Tier

df_groupby_flt_no = pd.merge(df_groupby_flt_no,
                            df_groupby_flt_no_ttl,
                            on = ['OPERATING_FLT_NO','DISCHARGE_STN_CD'],
                            how = 'left'
                           )

In [114]:
# a look at the data
df_groupby_flt_no.head(4)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,Lounge_eligibility_tier,pax_count,pax_count_ttl
0,5,HND,Not eligible,15825,21402
1,5,HND,Tier 1,220,21402
2,5,HND,Tier 2,888,21402
3,5,HND,Tier 3,4469,21402


In [115]:
# getting the percentage of pax elegible for each of the Tiers

df_groupby_flt_no['pax_eligible%'] = (df_groupby_flt_no['pax_count'] / df_groupby_flt_no['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_flt_no['pax_eligible%'] = df_groupby_flt_no['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_flt_no.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

In [116]:
# a look at the data
df_groupby_flt_no.head(4)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,Lounge_eligibility_tier,pax_eligible%
0,5,HND,Not eligible,73.9%
1,5,HND,Tier 1,1.0%
2,5,HND,Tier 2,4.1%
3,5,HND,Tier 3,20.9%


In [117]:
# dropping 'not eligible' rows - not needed anymore
mask = df_groupby_flt_no['Lounge_eligibility_tier'] == 'Not eligible'

df_groupby_flt_no = df_groupby_flt_no[~mask].copy()

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.

In [118]:
# using set_index to come up with the final lookup table
df_groupby_flt_no = df_groupby_flt_no.set_index(['OPERATING_FLT_NO','DISCHARGE_STN_CD','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

In [119]:
# final table
df_groupby_flt_no.head(4)

pax_eligible%              
Lounge_eligibility_tier                  Tier 1 Tier 2 Tier 3
OPERATING_FLT_NO DISCHARGE_STN_CD                            
5                HND                       1.0%   4.1%  20.9%
                 NRT                       3.6%   4.8%  19.1%
7                HND                       3.3%   5.2%  19.8%
9                BKK                       0.3%   4.2%  15.9%

<br>

**Feedback:** This analysis goes at a very granular level, we might come up with missing values if we apply this lounge eligibility profiles to a future schedule. Here are some ideas to further enhance your answer: 

- What would happen if we fly to a new destination in the future? How do we ensure we have a lounge eligibility profile for this new route?
- As you can see in the example above: Pax eligible for Tier 1 for the BKK flight is significantly different from the rest. What's the most used aircraft type for this route? And why it differs that much from the rest? Is it because of the route characteristics instead?
- For SH routes, we might change the time of departure for a specific flight number from one year to the next. Explore the possibility of using a clasification that takes that into account, like using Time of Day instead of flight number. 

### Ian  
First of all I would like to be able to split the data into 2019/23 data separately, which can be done by adding a year field into the table.

In [156]:
# Original merged table
headers_and_first_row(df_lounge_elig_flight_info_country_acft_typ)

{'Skew_Id': '2023-08-05618',
 'OPERATING_AIRLINE_CD': 'BA',
 'OPERATING_FLT_NO': 618,
 'UPLIFT_STN_CD': 'LHR',
 'DISCHARGE_STN_CD': 'OLB',
 'BOOKED_CABIN_CD': 'C',
 'TRAVEL_CABIN_CD': 'C',
 'BA_PAX_TIER': 'Gold For Life',
 'ONEWORLD_TIER': 'EMER',
 'Lounge_eligibility_tier': 'Tier 2',
 'pax': 2,
 'ACT_DEP_TML_CD': '3',
 'IATA_AC_TYP_CD': '319',
 'ACT_AC_TYP_CD': 'M4',
 'ROUTE_x': 'LHROLB',
 'GMT_PLND_DEP_DT': datetime.date(2023, 8, 5),
 'GMT_PLND_DEP_TIME': datetime.time(14, 25),
 'ROUTE_y': 'LHROLB',
 'COUNTRY_CD': 'IT',
 'COUNTRY_NM': 'Italy',
 'CORP_GEOG_CTRY_GRP_NM': 'MEDITERRANEAN',
 'CORP_GEOG_CONTINENT_NM': 'EUROPE EXC UK',
 'DEP_STN_CD': 'LHR',
 'ARR_STN_CD': 'OLB',
 'WB_NB_CAT': nan,
 'FIRST_SEATS_QTY': nan,
 'CLUB_SEATS_QTY': nan,
 'PREM_ECONOMY_SEATS_QTY': nan,
 'ECONOMY_SEATS_QTY': nan,
 'GMT_PLND_DEPT_YR': 2023}

In [157]:
# df_lounge_elig_flight_info_country_acft_typ['GMT_PLND_DEPT_YR'] = df_lounge_elig_flight_info_country_acft_typ['GMT_PLND_DEP_DT'].year

# headers_and_first_row(df_lounge_elig_flight_info_country_acft_typ)
df_lounge_elig_flight_info_country_acft_typ['GMT_PLND_DEPT_YR'] = [dts.year for dts in df_lounge_elig_flight_info_country_acft_typ['GMT_PLND_DEP_DT']]

df_lounge_elig_flight_info_country_acft_typ.head(10)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,ACT_DEP_TML_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,GMT_PLND_DEPT_YR
0,2023-08-05618,BA,618,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,3,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN,2023
1,2023-08-05618,BA,618,LHR,OLB,C,C,Gold Guest List,,Tier 1,1,3,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN,2023
2,2023-08-05618,BA,618,LHR,OLB,M,M,,,Not eligible,68,3,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN,2023
3,2023-08-05618,BA,618,LHR,OLB,M,M,Gold Guest List,EMER,Tier 1,4,3,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN,2023
4,2023-08-05618,BA,618,LHR,OLB,M,M,Silver,SAPP,Tier 3,6,3,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN,2023
5,2023-08-05618,BA,618,LHR,OLB,M,M,Premier,EMER,Tier 1,1,3,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN,2023
6,2023-08-05618,BA,618,LHR,OLB,M,M,Silver,,Tier 3,1,3,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN,2023
7,2023-08-05618,BA,618,LHR,OLB,C,C,Gold Guest List,EMER,Tier 1,1,3,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN,2023
8,2023-08-05618,BA,618,LHR,OLB,M,M,,EMER,Tier 2,2,3,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN,2023
9,2023-08-05618,BA,618,LHR,OLB,C,C,Silver,SAPP,Tier 3,2,3,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NaN,NaN,NaN,NaN,NaN,2023


In [158]:
# columns that you want to group by
list_groupby = ['GMT_PLND_DEPT_YR','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_year = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

# a look at the data
df_groupby_year

,GMT_PLND_DEPT_YR,Lounge_eligibility_tier,pax_count
0,2019,Not eligible,8105164
1,2019,Tier 1,196547
2,2019,Tier 2,635359
3,2019,Tier 3,1995324
4,2023,Not eligible,6856945
5,2023,Tier 1,88942
6,2023,Tier 2,466833
7,2023,Tier 3,1641743


### Compare by Year

In [159]:
# columns that you want to group by
list_groupby = ['GMT_PLND_DEPT_YR']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_year_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

# a look at the data
df_groupby_year_ttl

,GMT_PLND_DEPT_YR,pax_count_ttl
0,2019,10932394
1,2023,9054463


In [160]:
df_groupby_year = pd.merge(df_groupby_year,
                            df_groupby_year_ttl,
                            on = ['GMT_PLND_DEPT_YR'],
                            how = 'left'
                           )

df_groupby_year

,GMT_PLND_DEPT_YR,Lounge_eligibility_tier,pax_count,pax_count_ttl
0,2019,Not eligible,8105164,10932394
1,2019,Tier 1,196547,10932394
2,2019,Tier 2,635359,10932394
3,2019,Tier 3,1995324,10932394
4,2023,Not eligible,6856945,9054463
5,2023,Tier 1,88942,9054463
6,2023,Tier 2,466833,9054463
7,2023,Tier 3,1641743,9054463


In [161]:
# getting the percentage of pax elegible for each of the Tiers

df_groupby_year['pax_eligible%'] = (df_groupby_year['pax_count'] / df_groupby_year['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_year['pax_eligible%'] = df_groupby_year['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_year.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

In [162]:
df_groupby_year = df_groupby_year[df_groupby_year['Lounge_eligibility_tier'] != "Not eligible"]

df_groupby_year

,GMT_PLND_DEPT_YR,Lounge_eligibility_tier,pax_eligible%
1,2019,Tier 1,1.8%
2,2019,Tier 2,5.8%
3,2019,Tier 3,18.3%
5,2023,Tier 1,1.0%
6,2023,Tier 2,5.2%
7,2023,Tier 3,18.1%


Not much difference

### Compare by location and year

In [174]:
# columns that you want to group by
list_groupby = ['CORP_GEOG_CONTINENT_NM','CORP_GEOG_CTRY_GRP_NM','GMT_PLND_DEPT_YR','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_country_group_year = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

list_groupby = ['CORP_GEOG_CONTINENT_NM','CORP_GEOG_CTRY_GRP_NM','GMT_PLND_DEPT_YR']

df_groupby_country_group_year_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

df_groupby_country_group_year = pd.merge(df_groupby_country_group_year,
                                         df_groupby_country_group_year_ttl,
                                         on = list_groupby,
                                         how = "left")

df_groupby_country_group_year = df_groupby_country_group_year[df_groupby_country_group_year['Lounge_eligibility_tier'] != 'Not eligible']

df_groupby_country_group_year['pax_eligible%'] = (df_groupby_country_group_year['pax_count'] / df_groupby_country_group_year['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_country_group_year['pax_eligible%'] = df_groupby_country_group_year['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_country_group_year.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

df_groupby_country_group_year.head(6)


,CORP_GEOG_CONTINENT_NM,CORP_GEOG_CTRY_GRP_NM,GMT_PLND_DEPT_YR,Lounge_eligibility_tier,pax_eligible%
1,AFRICA,EAST AND CENTRAL AFRICA,2019,Tier 1,3.7%
2,AFRICA,EAST AND CENTRAL AFRICA,2019,Tier 2,4.0%
3,AFRICA,EAST AND CENTRAL AFRICA,2019,Tier 3,17.6%
5,AFRICA,EAST AND CENTRAL AFRICA,2023,Tier 1,0.2%
6,AFRICA,EAST AND CENTRAL AFRICA,2023,Tier 2,4.3%
7,AFRICA,EAST AND CENTRAL AFRICA,2023,Tier 3,16.0%


In [175]:
df_groupby_country_group_year = df_groupby_country_group_year.set_index(['CORP_GEOG_CONTINENT_NM','CORP_GEOG_CTRY_GRP_NM','GMT_PLND_DEPT_YR','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

df_groupby_country_group_year

pax_eligible%  \
Lounge_eligibility_tier                                                     Tier 1   
CORP_GEOG_CONTINENT_NM      CORP_GEOG_CTRY_GRP_NM   GMT_PLND_DEPT_YR                 
AFRICA                      EAST AND CENTRAL AFRICA 2019                      3.7%   
                                                    2023                      0.2%   
                            NORTH AFRICA            2019                      0.5%   
                                                    2023                      0.3%   
                            SOUTHERN AFRICA         2019                      3.8%   
                                                    2023                      2.4%   
                            WEST AFRICA             2019                      3.7%   
                                                    2023                      2.3%   
ASIA FAR EAST               CHINA                   2019                      4.6%   
                                                    2023                      0.2%   
                            FAR EAST HUBS           2019                      3.8%   
                                                    2023                      1.4%   
                            FAR EAST OTHER          2019                      0.4%   
                            JAPAN                   2019                      3.5%   
                                                    2023                      1.0%   
ASIA SOUTH EAST AND CENTRAL SOUTH ASIA              2019                      2.8%   
                                                    2023                      0.2%   
AUSTRALASIA PACIFIC         FAR EAST JSA HUBS       2019                      2.3%   
                                                    2023                      3.4%   
EUROPE EXC UK               BENELUX                 2019                      0.7%   
                                                    2023                      0.5%   
                            EASTERN EUROPE          2019                      0.5%   
                                                    2023                      0.3%   
                            IRELAND RO              2019                      0.6%   
                                                    2023                      0.6%   
                            MEDITERRANEAN           2019                      0.7%   
                                                    2023                      0.3%   
                            SCANDINAVIA             2019                      0.6%   
                                                    2023                      0.4%   
                            WEST EUROPE             2019                      0.5%   
                                                    2023                      0.4%   
MIDDLE EAST                 MIDDLE EAST OTHER       2019                      0.3%   
                                                    2023                      0.1%   
                            NORTHERN GULF           2019                      5.0%   
                                                    2023                      3.0%   
                            SAUDI ARABIA            2019                      4.0%   
                                                    2023                      3.5%   
                            SOUTHERN GULF           2019                      4.8%   
                                                    2023                      3.6%   
NORTH AMERICA               CANADA                  2019                      2.3%   
                                                    2023                      0.2%   
                            USA                     2019                      4.3%   
                                                    2023                      2.5%   
SOUTH AMERICA INC CARIBBEAN CARIBBEAN               2019                       NaN   
                           

### By continent

In [176]:
# columns that you want to group by
list_groupby = ['CORP_GEOG_CONTINENT_NM','GMT_PLND_DEPT_YR','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_cont_year = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

list_groupby = ['CORP_GEOG_CONTINENT_NM','GMT_PLND_DEPT_YR']

df_groupby_cont_year_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

df_groupby_cont_year = pd.merge(df_groupby_cont_year,
                                df_groupby_cont_year_ttl,
                                on = list_groupby,
                                how = "left")

df_groupby_cont_year = df_groupby_cont_year[df_groupby_cont_year['Lounge_eligibility_tier'] != 'Not eligible']

df_groupby_cont_year['pax_eligible%'] = (df_groupby_cont_year['pax_count'] / df_groupby_cont_year['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_cont_year['pax_eligible%'] = df_groupby_cont_year['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_cont_year.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

df_groupby_cont_year.head(6)

,CORP_GEOG_CONTINENT_NM,GMT_PLND_DEPT_YR,Lounge_eligibility_tier,pax_eligible%
1,AFRICA,2019,Tier 1,3.5%
2,AFRICA,2019,Tier 2,4.8%
3,AFRICA,2019,Tier 3,18.5%
5,AFRICA,2023,Tier 1,1.8%
6,AFRICA,2023,Tier 2,4.4%
7,AFRICA,2023,Tier 3,18.1%


In [177]:
df_groupby_cont_year = df_groupby_cont_year.set_index(['CORP_GEOG_CONTINENT_NM','GMT_PLND_DEPT_YR','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

df_groupby_cont_year

pax_eligible%              
Lounge_eligibility_tier                             Tier 1 Tier 2 Tier 3
CORP_GEOG_CONTINENT_NM      GMT_PLND_DEPT_YR                            
AFRICA                      2019                      3.5%   4.8%  18.5%
                            2023                      1.8%   4.4%  18.1%
ASIA FAR EAST               2019                      3.6%   4.8%  18.3%
                            2023                      1.0%   3.5%  18.3%
ASIA SOUTH EAST AND CENTRAL 2019                      2.8%   4.0%  19.1%
                            2023                      0.2%   3.0%  16.8%
AUSTRALASIA PACIFIC         2019                      2.3%   5.6%  18.3%
                            2023                      3.4%   7.4%  26.3%
EUROPE EXC UK               2019                      0.6%   6.4%  17.2%
                            2023                      0.4%   5.6%  18.0%
MIDDLE EAST                 2019                      4.0%   6.0%  20.2%
                            2023                      3.1%   5.0%  18.8%
NORTH AMERICA               2019                      4.1%   5.2%  20.5%
                            2023                      2.3%   5.1%  19.2%
SOUTH AMERICA INC CARIBBEAN 2019                      2.4%   3.6%  17.4%
                            2023                      1.2%   2.9%  16.5%
UK                          2019                      0.4%   5.7%  18.1%
                            2023                      0.3%   4.7%  16.5%

### By aircraft

In [190]:
list_groupby = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD','FIRST_SEATS_QTY','CLUB_SEATS_QTY','PREM_ECONOMY_SEATS_QTY','ECONOMY_SEATS_QTY','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_acft = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                mean_pax_count = ('pax','mean')).reset_index()

df_groupby_acft['mean_pax_count'] = df_groupby_acft['mean_pax_count'].map('{:,.1f}'.format)

df_groupby_acft = df_groupby_acft[df_groupby_acft['Lounge_eligibility_tier'] != 'Not eligible']

df_groupby_acft = df_groupby_acft.set_index(list_groupby,drop = True).unstack('Lounge_eligibility_tier')

df_groupby_acft.fillna(0, inplace=True)

df_groupby_acft


mean_pax_count  \
Lounge_eligibility_tier                                                                                      Tier 1   
IATA_AC_TYP_CD ACT_AC_TYP_CD FIRST_SEATS_QTY CLUB_SEATS_QTY PREM_ECONOMY_SEATS_QTY ECONOMY_SEATS_QTY                  
319            A4            0.0             16.0           0.0                    119.0                        1.2   
320            A3            0.0             24.0           0.0                    132.0                        1.2   
321            M6            0.0             23.0           0.0                    131.0                        1.3   
               V6            0.0             38.0           0.0                    160.0                        1.2   
               W6            0.0             42.0           0.0                    154.0                        1.3   
32A            H3            0.0             20.0           0.0                    150.0                        1.2   
32N            N3            0.0             20.0           0.0                    150.0                        1.2   
32Q            N6            0.0             32.0           0.0                    172.0                        1.2   
343            W9            0.0             45.0           0.0                    212.0                        1.3   
388            A8            14.0            97.0           55.0                   303.0                        2.2   
744            S4            14.0            86.0           30.0                   145.0                        2.1   
               V4            14.0            52.0           36.0                   235.0                        2.1   
777            K7            0.0             48.0           24.0                   203.0                          0   
               L7            0.0             48.0           24.0                   203.0                        1.3   
               P7            17.0            48.0           24.0                   127.0                        2.1   
               V7            14.0            48.0           40.0                   124.0                        2.1   
               W7            12.0            48.0           32.0                   127.0                        2.0   
77W            G7            14.0            56.0           44.0                   183.0                        2.1   
788            B8            0.0             35.0           25.0                   154.0                        1.2   
789            L8            8.0             42.0           39.0                   127.0                        1.6   

                                                                                                             \
Lounge_eligibility_tier                                                                              Tier 2   
IATA_AC_TYP_CD ACT_AC_TYP_CD FIRST_SEATS_QTY CLUB_SEATS_QTY PREM_ECONOMY_SEATS_QTY ECONOMY_SEATS_QTY          
319            A4            0.0             16.0           0.0                    119.0                2.1   
320            A3            0.0             24.0           0.0                    132.0                2.2   
321            M6            0.0             23.0           0.0                    131.0                2.0   
               V6            0.0             38.0           0.0                    160.0                2.5   
               W6            0.0             42.0           0.0                    154.0                2.4   
32A            H3            0.0             20.0           0.0                    150.0                2.2   
32N            N3            0.0             20.0           0.0                    150.0                2.2   
32Q            N6            0.0             32.0           0.0                    172.0                2.3   
343            W9            0.0             45.0           0.0                    212.0                2.2   
388            A8            14

<br><br><br>

In [163]:
#--------------------------------
#
# Your turn!!!
#
#--------------------------------

<u>What level of granularity do you use?</u>
- ... (your answer here)

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- ... (your answer here)

In [164]:
# your code here

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.